In [1]:
from typing import List
import pandas as pd
import json as js
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../genuine-essence-364222-54dfa19feae0.json"
"""imports for translator api"""
import six
from google.cloud import translate_v2 as translate


In [2]:
def get_translation(language_id: str, text: str) -> str:
    """Translates text into the target language using google api"""

    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    result = translate_client.translate(text, target_language=language_id)
    return result["translatedText"]

In [3]:
def get_audio(language_id: str,text: str):
    """Synthesizes speech from the input string of text."""
    from google.cloud import texttospeech

    client = texttospeech.TextToSpeechClient()

    input_text = texttospeech.SynthesisInput(text=text)

    # Note: the voice can also be specified by name.
    # Names of voices can be retrieved with client.list_voices().
    voice = texttospeech.VoiceSelectionParams(
        language_code=language_id
    #     name="en-US-Standard-C",
    #     ssml_gender=texttospeech.SsmlVoiceGender.FEMALE,
    )

    # we might be able to get another kind of response by changing this
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    response = client.synthesize_speech(
        request={"input": input_text, "voice": voice, "audio_config": audio_config}
    )

    return response.audio_content

In [6]:
def get_list_of_words() -> List[str]:
    """Get the list of words that we will use"""
    f = open("../data/words.txt","r")
    words = []
    for line in f.readlines():
        words.append(line.rstrip('\n'))
    return words

In [7]:
def get_list_of_languages() -> dict:
    """Get the list of languages"""
    f = open('../data/lan-dict.json')
    data = js.load(f)
    f.close()
    return data

In [8]:
def get_list_of_language_ids() -> List[str]:
    """Get the list of ids of languages"""
    ids = []
    f = open('../data/lan-dict.json')
    data = js.load(f)
    f.close()
    return data.values()


In [9]:
def get_table(type: str) -> pd.DataFrame:
    langs = get_list_of_languages()
    words = get_list_of_words()
    f = {}
    if type == 'text':
        for (lang,id) in langs.items():
            f[lang] = []
            for word in words:
                f[lang].append(get_translation(id,word))        
    elif type == 'audio':
        print("bye")
    else:
        raise ValueError()
    return pd.DataFrame(f)


In [8]:
d = get_table("text")
d.to_csv('../data/text.csv',index=False)
